In [1]:
] activate

 Activating environment at `~/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [2]:
using LinearAlgebra
using OrdinaryDiffEq #Gets the solvers
using Plots
plotly()
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

┌ Info: Precompiling ORCA [47be7bcc-f1a6-5447-8b36-7eeeff7534fd]
└ @ Base loading.jl:1260
[8384:0809/124157.968780:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command


In [ ]:
include(joinpath(pwd(), "data","make_data.jl"))

In [ ]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

## Experiment Parameters

In [ ]:
InitialLoad = 0.2
LoadStep = 0.5
CapSize = 2000e-6
StartTime=0
EndTime=0.05
TimeStep= 0.0001

## Instantiate ODE Models

In [ ]:
parameter_mapping = instantiate_parameters(omib_sys)
M_4th= instantiate_4th_order_model(omib_sys)
M_2nd= instantiate_2nd_order_model(omib_sys)
M_0th= instantiate_0th_order_model(omib_sys)

In [ ]:
_parameter_values = [x.second for x in parameter_mapping]; # Extract list of parameter values

In [ ]:
_parameter_values[39]=cap_size/_parameter_values[2]
_parameter_values[10]=InitialLoad

## Make copy of parameters and change value of load at t=0

In [54]:
disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simualte 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model for non-optimized gains

In [55]:
sim_4th_nonopt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (StartTime, EndTime))
sol_4th_nonopt = solve(sim_4th_nonopt, Rodas5(), saveat = TimeStep);

In [56]:
sim_2nd_nonopt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (StartTime, EndTime))
sol_2nd_nonopt = solve(sim_2nd_nonopt, Rodas5(), saveat = TimeStep);

In [57]:
sim_0th_nonopt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_0th_nonopt = solve(sim_0th_nonopt, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for all three models

In [58]:
plot(sol_4th_nonopt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_nonopt, vars=(0,13), label="2nd-order")
plot!(sol_0th_nonopt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [12]:
untuned_4th = [sol_4th_nonopt.t transpose(Array(sol_4th_nonopt))]
untuned_2nd = [sol_2nd_nonopt.t transpose(Array(sol_2nd_nonopt))]
untuned_0th = [sol_0th_nonopt.t transpose(Array(sol_0th_nonopt))];

In [13]:
fileName=string("results/dae_sim/untuned_4th_", cap_size/1e-6, "uF.csv")
writedlm( fileName,  untuned_4th, ',')
fileName=string("results/dae_sim/untuned_2nd_", cap_size/1e-6, "uF.csv")
writedlm( fileName,  untuned_2nd, ',')
fileName=string("results/dae_sim/untuned_0th_", cap_size/1e-6, "uF.csv")
writedlm( fileName,  untuned_0th, ',')

UndefVarError: UndefVarError: cap_size not defined

## Load optimal gains from parameter_search.ipynb

In [14]:
fileName=string("results/stable_gains/capSize_", cap_size/1e-6, "uF.csv") 
gains = readdlm(fileName, ',') # Loading set of stable gains 
fileName=string("results/optimal_gains/capSize_", cap_size/1e-6, "uF.csv")
optIndex = readdlm( fileName, ',') # Loading index of optimal set of gains

UndefVarError: UndefVarError: cap_size not defined

## Set controller gains to optimal values

In [15]:
_parameter_values[41]=gains[Int(optIndex[1]),1] #K_p^v
_parameter_values[42]=gains[Int(optIndex[1]),2] #K_i^v
_parameter_values[43]=gains[Int(optIndex[1]),3] #K_p^i
_parameter_values[44]=gains[Int(optIndex[1]),4] #K_i^i

UndefVarError: UndefVarError: optIndex not defined

## Re-calculate initial conditons

In [16]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067935237422023
 -0.040061818677185806
  0.2032828250035323
  0.05850652191227817
  0.2003183879749555
 -0.01599274198690052
  0.2023199540292933
  0.008076270122532621
  0.00027217172279205904
 -2.1729269003940947e-5
  0.07044778826693796
 -0.0027892516861153125
  1.084409521544636
  0.4294351687452164
  8.383249500865115e-19
  0.05466670651790573
 -1.411920284543435e-31
  4.600046756145388e-9
  0.0
  0.0
  1.4375146112954338e-13

## Make copy of parameters and change value of load at t=0

In [17]:
disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simulate 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model with optimized gains

In [18]:
sim_4th_opt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (StartTime, EndTime))
sol_4th_opt = solve(sim_4th_opt, Rodas5(), saveat = TimeStep);

In [19]:
sim_2nd_opt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (StartTime, EndTime))
sol_2nd_opt = solve(sim_2nd_opt, Rodas5(), saveat = TimeStep);

In [20]:
sim_0th_opt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_0th_opt = solve(sim_0th_opt, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for all three models

In [21]:
plot(sol_4th_opt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_opt, vars=(0,13), label="2nd-order")
plot!(sol_0th_opt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [22]:
tuned_4th = [sol_4th_opt.t transpose(Array(sol_4th_opt))]
tuned_2nd = [sol_2nd_opt.t transpose(Array(sol_2nd_opt))]
tuned_0th = [sol_0th_opt.t transpose(Array(sol_0th_opt))];

In [23]:
fileName=string("results/dae_sim/tuned_4th_", cap_size, "uF.csv")
writedlm( fileName,  tuned_4th, ',')
fileName=string("results/dae_sim/tuned_2nd_", cap_size, "uF.csv")
writedlm( fileName,  tuned_2nd, ',')
fileName=string("results/dae_sim/tuned_0th_", cap_size, "uF.csv")
writedlm( fileName,  tuned_0th, ',')

UndefVarError: UndefVarError: cap_size not defined

# Performance imporvement from increasing predictive gain

In [24]:
_parameter_values[45]=0 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k0 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k0 = solve(sim_k0, Rodas5(), saveat = TimeStep);

In [25]:
_parameter_values[45]=1 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k1 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k1 = solve(sim_k1, Rodas5(), saveat = TimeStep);

In [26]:
_parameter_values[45]=2 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k2 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k2 = solve(sim_k2, Rodas5(), saveat = TimeStep);

InterruptException: InterruptException:

In [27]:
_parameter_values[45]=5 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[10]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k5 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k5 = solve(sim_k5, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for various predictive gain

In [28]:
plot(sol_k0, vars=(0,13), xlims = (0,0.025), label="k_pred=0", ylabel="DC-link Voltage (p.u.)")
plot!(sol_k1, vars=(0,13), xlims = (0,0.025), label="k_pred=1")
plot!(sol_k2, vars=(0,13), xlims = (0,0.025), label="k_pred=2")
plot!(sol_k5, vars=(0,13), xlims = (0,0.025), label="k_pred=5")

UndefVarError: UndefVarError: sol_k2 not defined

## Save results to array and write to .csv file

In [29]:
kpred_0 = [sol_k0.t transpose(Array(sol_k0))]
kpred_1 = [sol_k1.t transpose(Array(sol_k1))]
kpred_2 = [sol_k2.t transpose(Array(sol_k2))];
kpred_5 = [sol_k5.t transpose(Array(sol_k5))];

UndefVarError: UndefVarError: sol_k2 not defined

In [30]:
writedlm( "results/dae_sim/kpred_0.csv",  kpred_0, ',')
writedlm( "results/dae_sim/kpred_1.csv",  kpred_1, ',')
writedlm( "results/dae_sim/kpred_2.csv",  kpred_2, ',')
writedlm( "results/dae_sim/kpred_5.csv",  kpred_5, ',')

UndefVarError: UndefVarError: kpred_2 not defined

## Calculating reduction in voltage nadir

In [31]:
nadir_reduction = 100*(minimum(kpred_5[:,14])-minimum(kpred_0[:,14]))/(1.1-minimum(kpred_0[:,14]))

UndefVarError: UndefVarError: kpred_5 not defined

## Sweeping DC_link Capacitor size and predictive gain to generate heatmap

In [32]:
heat_map_values=reshape([],0,3);
cap_range = 2000e-6:500e-6:7000e-6 # DC-link capacitor sizing
kpred_range = 0:0.25:5 # Range of predictive gain
for cap_size in cap_range
    for k in kpred_range
        _parameter_values[39]=cap_size/_parameter_values[2] # Adjusting DC-link capacitor size
        fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
        gains = readdlm(fileName, ',', Float64) # Loading set of stable gains
        fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
        optIndex = readdlm(fileName, ',', Float64) # Loading index of optimal gain
        _parameter_values[41]=gains[Int(optIndex[1]),1] #K_p^v
        _parameter_values[42]=gains[Int(optIndex[1]),2] #K_i^v
        _parameter_values[43]=gains[Int(optIndex[1]),3] #K_p^i
        _parameter_values[44]=gains[Int(optIndex[1]),4] #K_i^i
        _parameter_values[45]=k #K_pred
        M_0th(_parameter_values) #Initializing 0th order model

        disturbance=deepcopy(_parameter_values)
        disturbance[10]=0.7 # Step change in load from 0.2 p.u. to 0.7 p.u.

        sim = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
        sol = solve(sim, Rodas5(), saveat = TimeStep);
        solArray = Array(sol)
        nadir=100*maximum(_parameter_values[39].-solArray[13,:]) # Calculating nadir drop in % of Base Votlage 
        
        heat_map_values = [heat_map_values; [cap_size/1e3 k round(nadir, digits=3)]] # Concatenating results
    end
end
writedlm( "results/dae_sim/heat_map_matrix.csv",  heat_map_values, ',') # Writing array to .csv file

InterruptException: InterruptException: